In [8]:
# first define a graph of computations to perform
# then TensorFlow takes that graph and runs
# it efficiently using optimized C++ code.
# TensorFlow can train a network with millions of parameters on a training set composed of billions of 
# instances with millions of features each.
import tensorflow as tf

In [4]:
#construction phase
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [5]:
#execution phase
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [6]:
#execution phase 
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [7]:
print(result)

42


In [1]:
# let's start building the first neural network
from sklearn.datasets import fetch_openml

In [2]:
mnist = fetch_openml('mnist_784')
mnist['data'][:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [3]:
X, y = mnist["data"], mnist["target"]

In [17]:
# let's split the dataset before moving ahead
# mnist dataset is already split
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# shuffle the training set
import numpy as np
shuffled_indices = np.random.permutation(len(X_train))
X_train, y_train = X_train[shuffled_indices], y_train[shuffled_indices]

In [28]:
# scaling the feature set
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
X_train_scaled = X_train_scaled.reshape(-1, 784)

In [29]:
# extracting real valued feature columns
feature_columns = [tf.feature_column.numeric_column("X_train_scaled", shape=(784,))]
feature_columns

[NumericColumn(key='X_train_scaled', shape=(784,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [30]:
# creating the model
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300,100], n_classes = 10, feature_columns=feature_columns)

W0721 13:11:05.193293 139636469053248 estimator.py:1811] Using temporary folder as model directory: /tmp/tmplm4mh8je


In [32]:
input_fn = tf.estimator.inputs.numpy_input_fn(dict({"X_train_scaled": X_train_scaled}), np.array(y_train, np.int32), shuffle=True)
dnn_clf.train(input_fn = input_fn, steps=101)

W0721 13:12:32.203745 139636469053248 deprecation.py:323] From /home/dhruv/Documents/machine-learning-applications/my_env/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/canned/head.py:437: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0721 13:12:32.467827 139636469053248 deprecation.py:506] From /home/dhruv/Documents/machine-learning-applications/my_env/lib/python3.6/site-packages/tensorflow/python/training/adagrad.py:76: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0721 13:12:32.765969 139636469053248 deprecation.py:323] From /home/dhruv/Documents/machine-learning-applications/my_env/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispa

In [40]:
from sklearn.metrics import accuracy_score
test_input_fn = tf.estimator.inputs.numpy_input_fn(dict({"X_train_scaled": X_test}), np.array(y_test, np.int32), num_epochs=1, shuffle=False)
y_pred = list(dnn_clf.predict(input_fn = test_input_fn))

In [41]:
y_pred

[{'logits': array([ 100.66071 , -380.00217 ,  309.1811  ,  267.6807  ,  221.68718 ,
           51.424164, -304.2261  ,  885.20197 ,  237.6411  ,  153.06703 ],
        dtype=float32),
  'probabilities': array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32),
  'class_ids': array([7]),
  'classes': array([b'7'], dtype=object),
  'all_class_ids': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32),
  'all_classes': array([b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'8', b'9'],
        dtype=object)},
 {'logits': array([-619.24664  ,  448.65823  ,  605.8963   ,  607.2597   ,
           -2.5797236,  266.24365  ,  164.03113  , -102.09934  ,
          401.59668  , -231.93512  ], dtype=float32),
  'probabilities': array([0.        , 0.        , 0.20368774, 0.7963123 , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        dtype=float32),
  'class_ids': array([3]),
  'classes': array([b'3'], dtype=object),
  'all_class_ids': array([0, 1, 2, 3, 4, 5

In [44]:
accuracy_score = dnn_clf.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))


Test Accuracy: 80.659997%

